In [ ]:
from glob import glob as glob
import pandas as pd
import numpy as np
from scipy.stats import entropy
import seaborn as sns
import matplotlib.pyplot as plt
from kneed import KneeLocator

In [ ]:
# user input parameters

# global path to folder with .csv files
data_fpath = '/Users/sarahfisher/Documents/project/data/CSV_FB_20230404'

# list of columns of interest on .csv files 
feature_ref = ['mean_F_C2','mean_F_C3']
feature_name = ['Mean GFP','Mean RFP']

num_cells_ref = ['Number_cells']
num_cells_name = ['Number of Cells']

bin_max = 10000

In [ ]:
# output: list of .csv files (wells) from specified input data folder (experiment)
# requires: `global data_fpath`
# reference: https://docs.python.org/3/library/glob.html
def find_csv_files():
    csv_files = glob(data_fpath + '/Time_points_dfs_*.csv')
    csv_files.sort()
    return csv_files

# global list of .csv files
csv_files = find_csv_files()

# debug
print(csv_files)

In [ ]:
# output: dictionary mapping a well's number to its corresponding dataframe from each .csv file (well) in an experiment
# requires: `global csv_files`
# reference: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 
def create_dfs():
    dfs = {}
    for file in csv_files:
        df = pd.read_csv(file)
        i = file.split('_')[-1].split('.')[0]
        dfs[i] = df
    return dfs
        
# global dictionary to store dataframes, mapping number well to .csv file
dfs = create_dfs()

# debug
print(dfs)

In [ ]:
# output: dataframe containing only columns corresponding to features of interest, concatenated across all .csv files for an experiment
# requires: `global dfs`, `global col_means_name`, `global col_means_ref`
# reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html, https://pandas.pydata.org/docs/reference/api/pandas.concat.html
def create_df_features():
    df_features = pd.DataFrame(columns = feature_name)
    for i in dfs:
        df = dfs[i]
        df_cols = pd.DataFrame()
        df_cols[feature_name] = df[feature_ref]
        df_features = pd.concat([df_features, df_cols])
    return df_features

# global dataframe with columns for only features of interest
df_features = create_df_features()

# debug
print(df_features)

In [ ]:
# reference: https://dataindependent.com/pandas/pandas-mean-get-average-pd-dataframe-mean/#:~:text=Pandas%20Mean%20will%20return%20the,a%20scalar%20(single%20number).
def find_num_cells():
    df_num_cells = pd.DataFrame(columns = num_cells_name)
    for i in dfs:
        df = dfs[i]
        df_col = pd.DataFrame()
        df_col[num_cells_name] = df[num_cells_ref]
        df_num_cells = pd.concat([df_num_cells, df_col])
    avg = df_num_cells.mean()
    return avg[0] #float(avg[0])
    
num_cells = find_num_cells()

# debug
print(num_cells)
print(type(num_cells))

In [ ]:
# output: dictionary mapping each column of interest to a list of x values (bin sizes) for knee location
# requires: `global bin_max`, `global col_means_name`
def make_x_vals():
    x = {}
    x_vals_list = []
    for i in range(bin_max + 1):
        if i < 25:
            if i != 5 and i != 10:
                continue
            else: x_vals_list.append(i)
        elif i >= 25 and i < 200:
            if i % 25 != 0:
                continue
            else: x_vals_list.append(i) 
        elif i >= 25 and i < 100:
            if i % 25 != 0:
                continue
            else: x_vals_list.append(i) 
        elif i >= 100 and i < 500:
            if i % 100 != 0:
                continue
            else: x_vals_list.append(i) 
        elif i >= 500 and i < 2500:
            if i % 250 != 0:
                continue
            else: x_vals_list.append(i)
        elif i >= 2500 and i < bin_max:
            if i % 1250 != 0:
                continue
            else: x_vals_list.append(i) 
        elif i == bin_max: x_vals_list.append(i)
    for feature in feature_name:
        x[feature] = x_vals_list
    return x

# output: entropy for given column of dataframe
# requires: input parameters `col`, `col_min`, `col_max`, `i` (called in `make_y_vals()`)
# reference: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
def entropy_calc(col, col_min, col_max, i):
    ser = pd.Series(col)
    val_counts = ser.value_counts(bins = np.linspace(col_min, col_max, i))
    s = entropy(val_counts, base=2)
    return s

# output:  dictionary mapping each column of interest to a list of y values (calculated entropies) for knee location
# requires: `global x`, `global col_means_name`, `global df_means`; helper function `entropy_calc`
def make_y_vals():
    y = {}
    for feature in feature_name:
        df_col = df_features[feature]
        y[feature] = []
        for i in x[feature]:
            s = entropy_calc(df_col, df_col.min(), df_col.max(), i)
            y[feature].append(s)
    return y

x = make_x_vals()
y = make_y_vals()

# debug
print(x)
print(y)

In [ ]:
def plot_histogram():
    #for col in col_means_name:
        sns.histplot(df_features['Mean GFP'])
        plt.show()

plot_histogram()

# List of methods to estimate optimal number of bins
methods = ['auto', 'fd', 'doane', 'scott', 'stone', 'rice', 'sturges', 'sqrt']

# Calculate and print Shannon entropy for each method
for method in methods:
    hist, bin_edges = np.histogram(df_features['Mean GFP'], bins=method, density=False)
    probabilities = hist / len(df_features['Mean GFP'])
    probabilities = probabilities[probabilities > 0]  # exclude zero-probability bins
    shannon_entropy = -np.sum(probabilities * np.log2(probabilities))
    n_bins = len(bin_edges) - 1  # actual number of bins
    print(f'Shannon entropy for method "{method}" with {n_bins} bins: {shannon_entropy:.4f}')

In [ ]:
bin_range = [2,5,10,20,50,100,200,500]  # Geometric progression for the number of bins
#bin_range = x['Mean GFP']
entropies = []
for bins in bin_range:
    hist, _ = np.histogram(df_features['Mean GFP'], bins=bins, density=False)
    probabilities = hist / len(df_features['Mean GFP'])
    probabilities = probabilities[probabilities > 0]  # exclude zero-probability bins
    entropy = -np.sum(probabilities * np.log2(probabilities))
    entropies.append(entropy)

In [ ]:
knee_locator = KneeLocator(bin_range, entropies, curve="concave", direction="increasing")
optimal_bins = knee_locator.knee

plt.figure(figsize=(10, 6))
plt.plot(bin_range, entropies, label='Entropy')
plt.axvline(optimal_bins, color='red', linestyle='--', label='Knee point')

plt.xlabel('Number of bins')
plt.ylabel('Entropy')
plt.legend()
plt.title('Entropy as a function of number of bins')
plt.show()

In [ ]:
numpy_bins = [len(np.histogram(df_features['Mean GFP'], bins=method)[1])-1 for method in methods]
plt.figure(figsize=(10, 6))
plt.hist(numpy_bins, bins=30, alpha=0.5, color='grey', label='Numpy methods')
plt.axvline(optimal_bins, color='red', linestyle='--', label='Knee point')
plt.xlabel('Number of bins')
plt.ylabel('Count')
plt.legend()
plt.title('Number of bins suggested by Numpy methods')
plt.show()

In [ ]:
time_step = 1

def get_populations(time_step):
    dict_populations = {}
    for i in dfs:
        list_vals = []
        for j in range(len(dfs[i]['mean_F_C2'])):
            val = dfs[i]['mean_F_C2'][j]
            if dfs[i]['Time_interval'][j] == time_step:
                list_vals.append(val)
        dict_populations[i] = list_vals
    return dict_populations

dict_populations = get_populations(time_step)

In [ ]:
import math

from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


def plot_colortable(colors, *, ncols=4, sort_colors=True):

    cell_width = 212
    cell_height = 22
    swatch_width = 48
    margin = 12

    # Sort colors by hue, saturation, value and name.
    if sort_colors is True:
        names = sorted(
            colors, key=lambda c: tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(c))))
    else:
        names = list(colors)

    n = len(names)
    nrows = math.ceil(n / ncols)

    width = cell_width * 4 + 2 * margin
    height = cell_height * nrows + 2 * margin
    dpi = 72

    fig, ax = plt.subplots(figsize=(width / dpi, height / dpi), dpi=dpi)
    fig.subplots_adjust(margin/width, margin/height,
                        (width-margin)/width, (height-margin)/height)
    ax.set_xlim(0, cell_width * 4)
    ax.set_ylim(cell_height * (nrows-0.5), -cell_height/2.)
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.set_axis_off()

    for i, name in enumerate(names):
        row = i % nrows
        col = i // nrows
        y = row * cell_height

        swatch_start_x = cell_width * col
        text_pos_x = cell_width * col + swatch_width + 7

        ax.text(text_pos_x, y, name, fontsize=14,
                horizontalalignment='left',
                verticalalignment='center')

        ax.add_patch(
            Rectangle(xy=(swatch_start_x, y-9), width=swatch_width,
                      height=18, facecolor=colors[name], edgecolor='0.7')
        )

    return fig

plot_colortable(mcolors.CSS4_COLORS)
plt.show()

In [ ]:
colors = ['lightcoral', 'tomato', 'peachpuff', 'navajowhite', 'goldenrod', 'gold', 'yellowgreen', 'palegreen', 'mediumseagreen', 'paleturquoise', 'powderblue', 'skyblue', 'lightsteelblue', 'slateblue', 'mediumorchid', 'orchid']

pop_list = []
for pop in dict_populations:
    pop_list.append(dict_populations[pop])
    
for pop, color in zip(pop_list, colors):
    plt.hist(pop, bins=bin_edges, alpha=0.5, color=color, density=True)

plt.title('Cell Populations')
plt.xlabel('GFP')
plt.ylabel('Density')
plt.show()

shannon_entropy_list = []

# Calculate and print Shannon entropy for each population
for i, pop in enumerate(dict_populations):
    hist, _ = np.histogram(dict_populations[pop], bins=bin_edges, density=True)
    # Remove zero entries to avoid log2(0)
    hist = hist[hist > 0]
    shannon_entropy = -np.sum(hist * np.log2(hist))
    print(f'Shannon entropy for population {i+1}: {shannon_entropy:.4f}')
    shannon_entropy_list.append(shannon_entropy)
    
mean_entropy = sum(shannon_entropy_list) / len(shannon_entropy_list)
    
    

In [ ]:
# Draw a subset from each population for the joint distribution
subset_size = int(num_cells // 16)  # Size of subset from each population
joint_pop = np.concatenate([np.random.choice(dict_populations[pop], subset_size) for pop in dict_populations])

# Plot the histogram of the joint population
plt.hist(joint_pop, bins=bin_edges, alpha=0.5, color='grey', density=True)
plt.title('Joint Distribution')
plt.xlabel('GFP')
plt.ylabel('Density')
plt.show()

# Calculate and print Shannon entropy for the joint distribution
hist, _ = np.histogram(joint_pop, bins=bin_edges, density=True)
hist = hist[hist > 0]
shannon_entropy = -np.sum(hist * np.log2(hist))
print(f'Shannon entropy for the joint distribution: {shannon_entropy:.4f}')

In [ ]:
# Plot the histograms for individual populations
for pop, color in zip(pop_list, colors):
    plt.hist(pop, bins=bin_edges, alpha=0.5, color=color, density=True, label='Individual')

# Plot the histogram of the joint population
plt.hist(joint_pop, bins=bin_edges, alpha=0.5, color='grey', density=True, label='Joint')

plt.title('Individual and Joint Distributions')
plt.xlabel('GFP')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Calculate and print Shannon entropy for the joint distribution
hist, _ = np.histogram(joint_pop, bins=bin_edges, density=True)
hist = hist[hist > 0]
joint_shannon_entropy = -np.sum(hist * np.log2(hist))  # changed variable name
print(f'Shannon entropy for the joint distribution: {joint_shannon_entropy:.4f}')

# Mutual Information (MI)
MI = joint_shannon_entropy - mean_entropy  # use joint_shannon_entropy here
print(f'Mutual Information: {MI:.4f}')

In [ ]:
x_mi = []
y_mi = []
for timestep in dfs['01']['Time_interval']:
    if x_mi == [] or x_mi[-1] != timestep:
        print(timestep)
        x_mi.append(int(timestep))
        dict_populations_i = get_populations(timestep)
        #print(dict_populations_i)
        joint_pop_i = np.concatenate([np.random.choice(dict_populations_i[pop], subset_size) for pop in dict_populations_i])
        hist, _ = np.histogram(joint_pop_i, bins=bin_edges, density=True)
        hist = hist[hist > 0]
        joint_shannon_entropy_i = -np.sum(hist * np.log2(hist))  # changed variable name
        #print(f'Shannon entropy for the joint distribution: {joint_shannon_entropy_i:.4f}')
    
        shannon_entropy_list_i = []
    
        # Calculate and print Shannon entropy for each population
        for i, pop in enumerate(dict_populations_i):
            hist, _ = np.histogram(dict_populations_i[pop], bins=bin_edges, density=True)
            # Remove zero entries to avoid log2(0)
            hist = hist[hist > 0]
            shannon_entropy_i = -np.sum(hist * np.log2(hist))
            #print(f'Shannon entropy for population {i+1}: {shannon_entropy_i:.4f}')
            shannon_entropy_list_i.append(shannon_entropy_i)
    
        mean_entropy_i = sum(shannon_entropy_list_i) / len(shannon_entropy_list_i)

        # Mutual Information (MI)
        MI_i = joint_shannon_entropy_i - mean_entropy_i  # use joint_shannon_entropy here
        #print(f'Mutual Information: {MI_i:.4f}')
        y_mi.append(MI_i)
    
    
    

In [ ]:
print(bin_edges)

In [ ]:
x_mi_new = x_mi[:430]
print(len(x_mi_new))

In [ ]:
plt.scatter(np.array(x_mi_new), np.array(y_mi))
plt.show()

In [ ]:
avgy = [75.58575581395348, 72.20625, 74.5091911764706, 73.45454545454545, 74.50403225806451, 76.47426470588235, 70.76875, 70.76909722222223, 78.70833333333333, 82.49161585365853, 77.36583333333333, 83.75595238095238, 90.9225543478261, 94.341796875, 92.70436507936508, 94.38980263157895, 96.05555555555556, 92.66041666666666, 91.09114583333333, 85.91517857142857, 86.55208333333333, 86.921875, 88.45138888888889, 91.08035714285714, 90.99479166666667, 90.8125, 92.0, 92.44444444444444, 92.6875, 92.9, 92.96875, 94.109375, 94.33333333333333, 98.5875, 99.11778846153847, 100.69010416666667, 101.49553571428571, 99.58035714285714, 100.359375, 103.88392857142857, 107.63839285714286, 107.57986111111111, 107.8125, 110.009375, 115.98046875, 114.11458333333333, 114.76041666666667, 112.4875, 115.03645833333333, 114.128125, 112.5859375, 116.46875, 116.9375, 117.60526315789474, 119.53125, 119.43229166666667, 118.50520833333333, 117.1875, 119.00694444444444, 118.23611111111111, 117.17857142857143, 117.375, 117.51041666666667, 116.828125, 117.025, 119.10546875, 123.25, 128.125, 130.94375, 132.38839285714286, 136.09615384615384, 136.623046875, 136.74583333333334, 133.5, 135.14930555555554, 136.35714285714286, 136.96875, 139.03720238095238, 140.2878289473684, 142.1171875, 144.140625, 145.68125, 148.4375, 147.875, 146.203125, 146.3125, 148.1125, 148.70833333333334, 148.71484375, 151.88125, 151.625, 151.625, 151.83705357142858, 149.9375, 149.13541666666666, 152.76704545454547, 154.60416666666666, 155.171875, 156.15625, 156.0625, 156.0, 155.8375, 154.796875, 154.375, 148.5721153846154, 146.625, 145.25, 145.75, 145.5625, 145.3125, 146.7, 148.53571428571428, 149.125, 148.4125, 149.0, 151.609375, 151.265625, 150.97222222222223, 151.9375, 155.5, 155.73958333333334, 164.2130681818182, 160.94230769230768, 158.828125, 158.6875, 156.125, 157.5284090909091, 154.5078125, 158.00390625, 157.9375, 158.9375, 157.75, 156.75, 156.5, 158.2058823529412, 157.69642857142858, 158.125, 158.16826923076923, 161.81666666666666, 164.17083333333332, 165.875, 163.0, 164.42578125, 163.25, 161.61141304347825, 161.3566176470588, 160.03125, 165.25986842105263, 162.0625, 163.52604166666666, 163.64285714285714, 164.44485294117646, 164.40625, 165.125, 166.79017857142858, 165.9375, 165.375, 168.46428571428572, 169.80514705882354, 168.7, 169.1875, 170.21875, 169.72916666666666, 169.5625, 171.56041666666667, 175.65509259259258, 175.765625, 176.3465909090909, 176.6875, 178.62890625, 179.58680555555554, 180.60763888888889, 181.34375, 179.99305555555554, 180.00625, 178.63333333333333, 179.7775, 184.31119791666666, 184.625, 184.9, 184.83522727272728, 186.77083333333334, 189.32954545454547, 188.375, 186.04861111111111, 186.46875, 190.1640625, 192.2375, 192.625, 190.7890625, 190.26388888888889, 190.54166666666666, 191.875, 192.75, 194.4375, 195.0, 195.125, 195.0625, 195.4453125, 194.6875, 195.25892857142858, 197.75, 196.8125, 196.0625, 196.25, 198.05769230769232, 198.77083333333334, 195.9375, 196.8125, 198.43055555555554, 199.0625, 198.95625, 198.75, 199.0625, 199.1015625, 199.0, 198.875, 198.8125, 197.7734375, 196.375, 196.75, 196.71527777777777, 195.85714285714286, 194.16796875, 193.8625, 195.5, 196.0859375, 197.05357142857142, 199.3125, 200.5125, 201.4375, 202.67361111111111, 203.3125, 203.3806818181818, 201.1875, 202.74609375, 206.5, 208.859375, 207.82638888888889, 206.77864583333334, 208.55514705882354, 208.6898148148148, 208.15104166666666, 207.875, 207.4672619047619, 210.43055555555554, 214.33522727272728, 215.7421875, 212.54861111111111, 214.7875, 215.8359375, 218.5078125, 218.82986111111111, 218.25, 219.046875, 218.625, 216.896875, 218.08333333333334, 220.86197916666666, 224.546875, 223.071875, 223.61805555555554, 220.875, 223.03125, 223.96875, 224.375, 223.9375, 226.5625, 226.65625, 227.215625, 224.9375, 225.07954545454547, 225.16875, 225.125, 224.65178571428572, 225.35546875, 224.75, 222.64930555555554, 225.04861111111111, 226.22767857142858, 226.890625, 229.0625, 230.996875, 232.05357142857142, 230.9375, 232.85491071428572, 233.875, 232.47767857142858, 232.91964285714286, 233.70738636363637, 232.8359375, 236.09134615384616, 233.25, 233.83203125, 235.1875, 238.6171875, 239.95486111111111, 239.08564814814815, 241.3125, 240.65, 241.87291666666667, 240.087890625, 238.76, 238.709375, 238.43055555555554, 238.8138888888889, 239.97386363636363, 241.54583333333332, 240.34204545454546, 240.95138888888889, 242.5765625, 242.22916666666666, 241.35416666666666, 241.5975, 242.38888888888889, 242.8840909090909, 243.790625, 245.72625, 246.61647727272728, 245.515, 243.83875, 245.13888888888889, 246.9375, 246.94375, 245.5125, 246.06640625, 246.04125, 246.81944444444446, 250.0925, 248.759375, 247.30208333333334, 248.95, 252.23375, 252.08977272727273, 251.79017857142858, 255.65454545454546, 254.70625, 255.97544642857142, 256.46785714285716, 253.73035714285714, 252.646484375, 253.85, 254.76116071428572, 254.75714285714287, 254.675, 253.6015625, 254.4296875, 252.875, 251.9046875, 252.65555555555557, 254.6153846153846, 256.815625, 255.7140625, 256.2013888888889, 258.0885416666667, 258.5111607142857, 258.8097222222222, 261.0892857142857, 258.4171875, 259.628125, 263.3359375, 264.00625, 264.22222222222223, 263.52777777777777, 261.5982142857143, 263.111328125, 260.66015625, 258.8125, 255.725, 254.88616071428572, 256.146875, 255.1296875, 256.16796875, 253.46527777777777, 254.84722222222223, 251.51785714285714, 252.5984375, 253.22222222222223, 252.66927083333334, 251.140625, 251.89375, 252.11160714285714, 250.85546875, 254.84114583333334, 252.925, 255.6578125, 255.6534090909091, 259.1328125, 259.626953125, 259.26785714285717, 259.65625, 258.80078125, 257.0453125, 259.8125, 258.765625, 257.23579545454544, 255.05034722222223, 256.3863636363636, 254.6, 254.59375, 254.24305555555554, 252.15625, 253.6796875, 254.53571428571428, 253.26785714285714, 253.84635416666666, 255.32386363636363, 254.28794642857142, 255.24330357142858, 264.5980392156869, 269.4666666666665, 270.0320512820514, 266.7897435897437, 262.80606060606044, 266.0369047619051, 262.1000000000001, 262.53333333333353, 263.0444444444445, 264.3400000000001, 259.8592592592594, 266.0722222222221, 266.87826086956466, 269.9089743589744, 271.6866666666668, 271.72407407407417, 271.1375000000002, 274.0456790123458, 273.0124999999995, 269.6249999999998, 269.8166666666668]
stdy = [2.032754284479767, 1.077486223809845, 2.2241226558918816, 2.153566497005559, 1.653665854102313, 2.0314687465393333, 0.9474678424622125, 0.85528628913433, 4.120150505864103, 2.001518149870457, 2.482159118366284, 1.6791465158055596, 2.3154426335881477, 1.1981415026209277, 1.4984369480735509, 1.504204237687584, 0.7916362079910628, 0.9300654285526847, 2.6657904138952806, 0.2605016868016993, 0.4003308093869924, 0.6177311687740874, 1.49316860037164, 0.09278843611976129, 0.1830342019875581, 0.15811388300841897, 0.3612045819199972, 0.7605989761162815, 0.701560760020114, 0.3657184709581949, 0.40625, 0.24356964481437338, 0.55979286843935, 0.5736396952791883, 0.263646148824406, 0.5313967832351254, 0.5316250176945924, 0.8931249598334776, 0.15546678704790937, 1.8308579454251854, 0.6818675818915971, 0.8998952378156082, 0.4050462936504913, 1.9937671433432238, 1.010617160896468, 0.1854634772371339, 0.16304662335935965, 1.4604954210814904, 0.3675105700482943, 1.288239320497166, 0.39397005259759277, 1.016369919123938, 0.0, 0.8730129297351584, 0.21875, 0.3797085756610661, 0.5391316883939911, 1.008944374086104, 0.5761796809080982, 0.7317085325067694, 0.45060546003376933, 0.0, 0.16304662335935965, 0.18944305707784595, 1.1809688818931683, 0.5857682047093692, 2.236747235260266, 0.0, 0.6545812115391031, 1.1650075134425317, 0.8437465757107808, 0.5054595427581068, 0.6995037923811103, 0.0, 0.8047544507986563, 0.3019916879515672, 0.7686483672655527, 0.8755376243548861, 0.9100746736012771, 0.556774243831151, 0.5122998847110938, 2.9432722542265775, 0.0, 0.0, 0.7027776920015318, 0.2651650429449553, 0.525, 0.25108272485820726, 0.5270832690011489, 0.517619249545455, 0.1875, 0.0, 0.7706131758365926, 0.0, 0.3825909931709782, 1.6836215761180562, 0.5073248192452466, 0.20432338797846908, 0.06987712429686843, 0.0, 0.0, 0.2, 0.7307089344431201, 0.879452954966893, 2.3921252713036307, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45868357829771933, 0.680448531544916, 0.0, 0.030618621784789725, 0.0, 0.981145496027475, 0.20669932117692114, 0.09820927516479827, 0.0, 0.0, 0.3651780863116271, 1.9118220622285866, 0.19985201625794738, 0.12401959270615269, 0.0, 0.16237976320958225, 0.9775039367086925, 1.0128266737669136, 0.5678859980981548, 0.5537514108334172, 0.0, 0.5031152949374527, 0.0, 0.0, 0.07953421931170565, 0.4799885734184147, 0.0, 0.5062022860851872, 0.5251322584729384, 0.44631982802569836, 0.0, 0.0, 0.43313601965599385, 0.15309310892394862, 1.29785521779265, 0.24607647839237515, 0.7686483672655527, 0.2614666045952323, 0.0, 1.1228156842349009, 0.12371791482634838, 0.39868946210958706, 0.16237976320958225, 0.5, 0.36413359043580895, 0.3535533905932738, 0.0, 0.8130884839013902, 0.3045928702530585, 0.400975373308636, 0.1767766952966369, 0.3788861141556919, 0.2845104225780763, 0.0, 0.7092181973052363, 1.3528701454003267, 0.7810937343718742, 0.2971976552414095, 0.75, 1.601500570448989, 0.2518498306359526, 0.23367307442210056, 0.06987712429686843, 0.6158669113283615, 0.31842303491843466, 0.36505326856349185, 0.23323807579381203, 1.2012966074013498, 0.0, 0.6434768838116876, 0.6419700277496806, 1.0248644896874037, 0.18672359914948844, 0.0, 0.6678230711206281, 0.9084026158592896, 1.316191794475163, 0.0375, 0.0, 0.5710611524554529, 0.2296921584519615, 0.11785113019775792, 1.403121520040228, 0.0, 0.3535533905932738, 0.0, 0.0, 0.0, 0.09077304717673633, 0.0, 1.3030820999398591, 0.0, 0.7071067811865476, 0.0, 0.0, 0.7477777728932147, 0.860116473055959, 0.0, 0.0, 0.49690399499995325, 0.0, 0.028641098093474003, 0.0, 0.4592793267718459, 0.2687091175299975, 0.0, 0.0, 0.0, 0.9677098750884741, 0.0, 0.0, 0.09820927516479826, 1.0164463907684076, 0.16641725315734995, 0.225, 0.0, 0.2118037767457181, 0.5905019915638019, 0.5625, 0.8573214099741123, 0.0, 0.9938079899999066, 0.0, 0.773520648861345, 0.6634034720037774, 1.386399611164089, 1.3110110602126894, 0.8179405190527813, 0.1552824984374854, 0.9746091575511893, 1.3073631420674392, 1.3694629127545408, 0.8164799690932751, 0.0, 0.3190498400775571, 2.4401595968347425, 0.5785419118799026, 0.24494399072798254, 0.44172344149414583, 1.1465641608736947, 0.22736925329461324, 1.0562488443041014, 0.979788268022393, 0.5412658773652742, 0.5392323102105436, 0.0, 0.8026506381203469, 0.914058790353346, 1.0581988872525991, 0.8812001315677387, 0.6316134473513052, 0.3637722459363554, 0.5303300858899106, 0.8149463909928725, 0.5385527481129402, 0.0, 0.46770717334674267, 0.0, 0.59375, 1.1022836735046926, 0.0, 0.26191887662763885, 0.3075228650035636, 0.0, 0.5216914699282693, 0.07564420598061361, 0.25, 0.8052935996490985, 0.12422599874998831, 0.6865281860601952, 1.3058798817751194, 0.0, 1.251635648611448, 0.27836530835928386, 0.15309310892394862, 0.7325068509378724, 0.0, 0.24731498186543824, 0.1694077317947346, 0.23032397793987844, 0.053559801565633156, 1.8249926541961266, 0.0, 1.1292711348026379, 0.0, 0.677078325301992, 1.4628732245097844, 0.6180512206811499, 1.3096447116807417, 0.10777097630314633, 0.7939358924931463, 1.11627031570521, 1.2054796763114672, 0.8622452069307199, 0.6246140783824683, 0.8267961181498461, 0.6447659579226154, 0.6019932170160502, 1.2032068033261145, 0.6676930756042603, 0.43983473014730207, 1.216780816562932, 1.1231466214761692, 1.2597842077117811, 0.7194457850695358, 1.2502510078558735, 1.1660242837129653, 0.5458493954379724, 0.5665245717465061, 1.4406248644251562, 0.7782281558129338, 1.2530441328083508, 0.2851684111665819, 0.914146630743668, 0.6950157372318989, 0.2910352866680216, 0.7758674580751534, 1.2139397395909008, 0.5562485955038449, 1.0406343843420705, 0.6022779385614069, 1.1156906493289258, 0.40750958577682567, 0.6117648227440896, 1.8972058788343646, 1.0861900774254571, 0.8110804907652507, 2.387265045803632, 1.6412900134233637, 0.5085807081817928, 0.808525340554246, 0.9088110364646766, 0.24324212854579652, 0.44840277766803377, 0.6534962063139056, 1.105047678199339, 1.4176060572647642, 1.5026019100214134, 0.5036948442199404, 0.6562496325689741, 1.3628938549143559, 0.7079280396869445, 0.46538112321381275, 1.0811587862047156, 0.7498733978794169, 0.09157228346770543, 1.2036515302083237, 0.5837811085099117, 0.5452901141536952, 0.5067802007527524, 0.6483433666613317, 0.6348904925260734, 0.3334779778761936, 0.34246744460938755, 1.0359913422536358, 0.24040898126210755, 1.4388527517473557, 1.6688787403523362, 0.32739502134271986, 0.023937065390097333, 0.21494457861272054, 0.30051632001232476, 0.6393733007980842, 1.1603665874558569, 0.6555790956225527, 0.44284274170133014, 1.2434506156634246, 0.4183415411619176, 0.26268542971135433, 0.49286907173710953, 0.39533411502171173, 0.49786293934932635, 0.4084787122953135, 1.8432184232521038, 1.0444421357835005, 0.16535207465813667, 0.7388199072655669, 1.4330902129990806, 0.438749436598196, 0.5030202150358313, 1.197083043276447, 2.472230413568573, 0.6204502365973841, 0.8385254915624212, 0.6816154409746011, 0.5930563586412285, 0.8758452569712019, 1.1514621947551789, 0.4199567077254943, 0.5953924651018015, 0.3693961434591178, 1.0190779976418771, 0.7258901458941636, 0.5562471336775497, 0.3467229971240643, 0.39176050238971116, 0.3511023446303922, 1.083348408369079, 1.142881120702706, 7.951266665139676, 1.1368683772161603e-13, 1.555057389471424, 0.6594979462374291, 0.49966012323492465, 0.170861330043721, 0.7078920193865202, 0.0, 0.32810717911629494, 0.5800000000000012, 0.5447341129140679, 0.9733326991372867, 1.3512423589509706, 0.2932368274098105, 0.7284992183325364, 0.2902331024750524, 0.1301041249666259, 0.8677414109204393, 1.4531992961890707, 0.792280463808948, 0.6386472973170435, 5.684341886080802e-14]
for i in range(len(avgy)):
    avgy[i] = avgy[i]/100
    stdy[i] = stdy[i]/100

plt.scatter(np.array(x_mi_new), np.array(y_mi), 3)
plt.plot(np.array(x_mi_new), np.array(avgy), color='red', linewidth=1)
plt.show()


In [ ]:
avgy = [75.58575581395348, 72.20625, 74.5091911764706, 73.45454545454545, 74.50403225806451, 76.47426470588235, 70.76875, 70.76909722222223, 78.70833333333333, 82.49161585365853, 77.36583333333333, 83.75595238095238, 90.9225543478261, 94.341796875, 92.70436507936508, 94.38980263157895, 96.05555555555556, 92.66041666666666, 91.09114583333333, 85.91517857142857, 86.55208333333333, 86.921875, 88.45138888888889, 91.08035714285714, 90.99479166666667, 90.8125, 92.0, 92.44444444444444, 92.6875, 92.9, 92.96875, 94.109375, 94.33333333333333, 98.5875, 99.11778846153847, 100.69010416666667, 101.49553571428571, 99.58035714285714, 100.359375, 103.88392857142857, 107.63839285714286, 107.57986111111111, 107.8125, 110.009375, 115.98046875, 114.11458333333333, 114.76041666666667, 112.4875, 115.03645833333333, 114.128125, 112.5859375, 116.46875, 116.9375, 117.60526315789474, 119.53125, 119.43229166666667, 118.50520833333333, 117.1875, 119.00694444444444, 118.23611111111111, 117.17857142857143, 117.375, 117.51041666666667, 116.828125, 117.025, 119.10546875, 123.25, 128.125, 130.94375, 132.38839285714286, 136.09615384615384, 136.623046875, 136.74583333333334, 133.5, 135.14930555555554, 136.35714285714286, 136.96875, 139.03720238095238, 140.2878289473684, 142.1171875, 144.140625, 145.68125, 148.4375, 147.875, 146.203125, 146.3125, 148.1125, 148.70833333333334, 148.71484375, 151.88125, 151.625, 151.625, 151.83705357142858, 149.9375, 149.13541666666666, 152.76704545454547, 154.60416666666666, 155.171875, 156.15625, 156.0625, 156.0, 155.8375, 154.796875, 154.375, 148.5721153846154, 146.625, 145.25, 145.75, 145.5625, 145.3125, 146.7, 148.53571428571428, 149.125, 148.4125, 149.0, 151.609375, 151.265625, 150.97222222222223, 151.9375, 155.5, 155.73958333333334, 164.2130681818182, 160.94230769230768, 158.828125, 158.6875, 156.125, 157.5284090909091, 154.5078125, 158.00390625, 157.9375, 158.9375, 157.75, 156.75, 156.5, 158.2058823529412, 157.69642857142858, 158.125, 158.16826923076923, 161.81666666666666, 164.17083333333332, 165.875, 163.0, 164.42578125, 163.25, 161.61141304347825, 161.3566176470588, 160.03125, 165.25986842105263, 162.0625, 163.52604166666666, 163.64285714285714, 164.44485294117646, 164.40625, 165.125, 166.79017857142858, 165.9375, 165.375, 168.46428571428572, 169.80514705882354, 168.7, 169.1875, 170.21875, 169.72916666666666, 169.5625, 171.56041666666667, 175.65509259259258, 175.765625, 176.3465909090909, 176.6875, 178.62890625, 179.58680555555554, 180.60763888888889, 181.34375, 179.99305555555554, 180.00625, 178.63333333333333, 179.7775, 184.31119791666666, 184.625, 184.9, 184.83522727272728, 186.77083333333334, 189.32954545454547, 188.375, 186.04861111111111, 186.46875, 190.1640625, 192.2375, 192.625, 190.7890625, 190.26388888888889, 190.54166666666666, 191.875, 192.75, 194.4375, 195.0, 195.125, 195.0625, 195.4453125, 194.6875, 195.25892857142858, 197.75, 196.8125, 196.0625, 196.25, 198.05769230769232, 198.77083333333334, 195.9375, 196.8125, 198.43055555555554, 199.0625, 198.95625, 198.75, 199.0625, 199.1015625, 199.0, 198.875, 198.8125, 197.7734375, 196.375, 196.75, 196.71527777777777, 195.85714285714286, 194.16796875, 193.8625, 195.5, 196.0859375, 197.05357142857142, 199.3125, 200.5125, 201.4375, 202.67361111111111, 203.3125, 203.3806818181818, 201.1875, 202.74609375, 206.5, 208.859375, 207.82638888888889, 206.77864583333334, 208.55514705882354, 208.6898148148148, 208.15104166666666, 207.875, 207.4672619047619, 210.43055555555554, 214.33522727272728, 215.7421875, 212.54861111111111, 214.7875, 215.8359375, 218.5078125, 218.82986111111111, 218.25, 219.046875, 218.625, 216.896875, 218.08333333333334, 220.86197916666666, 224.546875, 223.071875, 223.61805555555554, 220.875, 223.03125, 223.96875, 224.375, 223.9375, 226.5625, 226.65625, 227.215625, 224.9375, 225.07954545454547, 225.16875, 225.125, 224.65178571428572, 225.35546875, 224.75, 222.64930555555554, 225.04861111111111, 226.22767857142858, 226.890625, 229.0625, 230.996875, 232.05357142857142, 230.9375, 232.85491071428572, 233.875, 232.47767857142858, 232.91964285714286, 233.70738636363637, 232.8359375, 236.09134615384616, 233.25, 233.83203125, 235.1875, 238.6171875, 239.95486111111111, 239.08564814814815, 241.3125, 240.65, 241.87291666666667, 240.087890625, 238.76, 238.709375, 238.43055555555554, 238.8138888888889, 239.97386363636363, 241.54583333333332, 240.34204545454546, 240.95138888888889, 242.5765625, 242.22916666666666, 241.35416666666666, 241.5975, 242.38888888888889, 242.8840909090909, 243.790625, 245.72625, 246.61647727272728, 245.515, 243.83875, 245.13888888888889, 246.9375, 246.94375, 245.5125, 246.06640625, 246.04125, 246.81944444444446, 250.0925, 248.759375, 247.30208333333334, 248.95, 252.23375, 252.08977272727273, 251.79017857142858, 255.65454545454546, 254.70625, 255.97544642857142, 256.46785714285716, 253.73035714285714, 252.646484375, 253.85, 254.76116071428572, 254.75714285714287, 254.675, 253.6015625, 254.4296875, 252.875, 251.9046875, 252.65555555555557, 254.6153846153846, 256.815625, 255.7140625, 256.2013888888889, 258.0885416666667, 258.5111607142857, 258.8097222222222, 261.0892857142857, 258.4171875, 259.628125, 263.3359375, 264.00625, 264.22222222222223, 263.52777777777777, 261.5982142857143, 263.111328125, 260.66015625, 258.8125, 255.725, 254.88616071428572, 256.146875, 255.1296875, 256.16796875, 253.46527777777777, 254.84722222222223, 251.51785714285714, 252.5984375, 253.22222222222223, 252.66927083333334, 251.140625, 251.89375, 252.11160714285714, 250.85546875, 254.84114583333334, 252.925, 255.6578125, 255.6534090909091, 259.1328125, 259.626953125, 259.26785714285717, 259.65625, 258.80078125, 257.0453125, 259.8125, 258.765625, 257.23579545454544, 255.05034722222223, 256.3863636363636, 254.6, 254.59375, 254.24305555555554, 252.15625, 253.6796875, 254.53571428571428, 253.26785714285714, 253.84635416666666, 255.32386363636363, 254.28794642857142, 255.24330357142858, 264.5980392156869, 269.4666666666665, 270.0320512820514, 266.7897435897437, 262.80606060606044, 266.0369047619051, 262.1000000000001, 262.53333333333353, 263.0444444444445, 264.3400000000001, 259.8592592592594, 266.0722222222221, 266.87826086956466, 269.9089743589744, 271.6866666666668, 271.72407407407417, 271.1375000000002, 274.0456790123458, 273.0124999999995, 269.6249999999998, 269.8166666666668]
stdy = [2.032754284479767, 1.077486223809845, 2.2241226558918816, 2.153566497005559, 1.653665854102313, 2.0314687465393333, 0.9474678424622125, 0.85528628913433, 4.120150505864103, 2.001518149870457, 2.482159118366284, 1.6791465158055596, 2.3154426335881477, 1.1981415026209277, 1.4984369480735509, 1.504204237687584, 0.7916362079910628, 0.9300654285526847, 2.6657904138952806, 0.2605016868016993, 0.4003308093869924, 0.6177311687740874, 1.49316860037164, 0.09278843611976129, 0.1830342019875581, 0.15811388300841897, 0.3612045819199972, 0.7605989761162815, 0.701560760020114, 0.3657184709581949, 0.40625, 0.24356964481437338, 0.55979286843935, 0.5736396952791883, 0.263646148824406, 0.5313967832351254, 0.5316250176945924, 0.8931249598334776, 0.15546678704790937, 1.8308579454251854, 0.6818675818915971, 0.8998952378156082, 0.4050462936504913, 1.9937671433432238, 1.010617160896468, 0.1854634772371339, 0.16304662335935965, 1.4604954210814904, 0.3675105700482943, 1.288239320497166, 0.39397005259759277, 1.016369919123938, 0.0, 0.8730129297351584, 0.21875, 0.3797085756610661, 0.5391316883939911, 1.008944374086104, 0.5761796809080982, 0.7317085325067694, 0.45060546003376933, 0.0, 0.16304662335935965, 0.18944305707784595, 1.1809688818931683, 0.5857682047093692, 2.236747235260266, 0.0, 0.6545812115391031, 1.1650075134425317, 0.8437465757107808, 0.5054595427581068, 0.6995037923811103, 0.0, 0.8047544507986563, 0.3019916879515672, 0.7686483672655527, 0.8755376243548861, 0.9100746736012771, 0.556774243831151, 0.5122998847110938, 2.9432722542265775, 0.0, 0.0, 0.7027776920015318, 0.2651650429449553, 0.525, 0.25108272485820726, 0.5270832690011489, 0.517619249545455, 0.1875, 0.0, 0.7706131758365926, 0.0, 0.3825909931709782, 1.6836215761180562, 0.5073248192452466, 0.20432338797846908, 0.06987712429686843, 0.0, 0.0, 0.2, 0.7307089344431201, 0.879452954966893, 2.3921252713036307, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45868357829771933, 0.680448531544916, 0.0, 0.030618621784789725, 0.0, 0.981145496027475, 0.20669932117692114, 0.09820927516479827, 0.0, 0.0, 0.3651780863116271, 1.9118220622285866, 0.19985201625794738, 0.12401959270615269, 0.0, 0.16237976320958225, 0.9775039367086925, 1.0128266737669136, 0.5678859980981548, 0.5537514108334172, 0.0, 0.5031152949374527, 0.0, 0.0, 0.07953421931170565, 0.4799885734184147, 0.0, 0.5062022860851872, 0.5251322584729384, 0.44631982802569836, 0.0, 0.0, 0.43313601965599385, 0.15309310892394862, 1.29785521779265, 0.24607647839237515, 0.7686483672655527, 0.2614666045952323, 0.0, 1.1228156842349009, 0.12371791482634838, 0.39868946210958706, 0.16237976320958225, 0.5, 0.36413359043580895, 0.3535533905932738, 0.0, 0.8130884839013902, 0.3045928702530585, 0.400975373308636, 0.1767766952966369, 0.3788861141556919, 0.2845104225780763, 0.0, 0.7092181973052363, 1.3528701454003267, 0.7810937343718742, 0.2971976552414095, 0.75, 1.601500570448989, 0.2518498306359526, 0.23367307442210056, 0.06987712429686843, 0.6158669113283615, 0.31842303491843466, 0.36505326856349185, 0.23323807579381203, 1.2012966074013498, 0.0, 0.6434768838116876, 0.6419700277496806, 1.0248644896874037, 0.18672359914948844, 0.0, 0.6678230711206281, 0.9084026158592896, 1.316191794475163, 0.0375, 0.0, 0.5710611524554529, 0.2296921584519615, 0.11785113019775792, 1.403121520040228, 0.0, 0.3535533905932738, 0.0, 0.0, 0.0, 0.09077304717673633, 0.0, 1.3030820999398591, 0.0, 0.7071067811865476, 0.0, 0.0, 0.7477777728932147, 0.860116473055959, 0.0, 0.0, 0.49690399499995325, 0.0, 0.028641098093474003, 0.0, 0.4592793267718459, 0.2687091175299975, 0.0, 0.0, 0.0, 0.9677098750884741, 0.0, 0.0, 0.09820927516479826, 1.0164463907684076, 0.16641725315734995, 0.225, 0.0, 0.2118037767457181, 0.5905019915638019, 0.5625, 0.8573214099741123, 0.0, 0.9938079899999066, 0.0, 0.773520648861345, 0.6634034720037774, 1.386399611164089, 1.3110110602126894, 0.8179405190527813, 0.1552824984374854, 0.9746091575511893, 1.3073631420674392, 1.3694629127545408, 0.8164799690932751, 0.0, 0.3190498400775571, 2.4401595968347425, 0.5785419118799026, 0.24494399072798254, 0.44172344149414583, 1.1465641608736947, 0.22736925329461324, 1.0562488443041014, 0.979788268022393, 0.5412658773652742, 0.5392323102105436, 0.0, 0.8026506381203469, 0.914058790353346, 1.0581988872525991, 0.8812001315677387, 0.6316134473513052, 0.3637722459363554, 0.5303300858899106, 0.8149463909928725, 0.5385527481129402, 0.0, 0.46770717334674267, 0.0, 0.59375, 1.1022836735046926, 0.0, 0.26191887662763885, 0.3075228650035636, 0.0, 0.5216914699282693, 0.07564420598061361, 0.25, 0.8052935996490985, 0.12422599874998831, 0.6865281860601952, 1.3058798817751194, 0.0, 1.251635648611448, 0.27836530835928386, 0.15309310892394862, 0.7325068509378724, 0.0, 0.24731498186543824, 0.1694077317947346, 0.23032397793987844, 0.053559801565633156, 1.8249926541961266, 0.0, 1.1292711348026379, 0.0, 0.677078325301992, 1.4628732245097844, 0.6180512206811499, 1.3096447116807417, 0.10777097630314633, 0.7939358924931463, 1.11627031570521, 1.2054796763114672, 0.8622452069307199, 0.6246140783824683, 0.8267961181498461, 0.6447659579226154, 0.6019932170160502, 1.2032068033261145, 0.6676930756042603, 0.43983473014730207, 1.216780816562932, 1.1231466214761692, 1.2597842077117811, 0.7194457850695358, 1.2502510078558735, 1.1660242837129653, 0.5458493954379724, 0.5665245717465061, 1.4406248644251562, 0.7782281558129338, 1.2530441328083508, 0.2851684111665819, 0.914146630743668, 0.6950157372318989, 0.2910352866680216, 0.7758674580751534, 1.2139397395909008, 0.5562485955038449, 1.0406343843420705, 0.6022779385614069, 1.1156906493289258, 0.40750958577682567, 0.6117648227440896, 1.8972058788343646, 1.0861900774254571, 0.8110804907652507, 2.387265045803632, 1.6412900134233637, 0.5085807081817928, 0.808525340554246, 0.9088110364646766, 0.24324212854579652, 0.44840277766803377, 0.6534962063139056, 1.105047678199339, 1.4176060572647642, 1.5026019100214134, 0.5036948442199404, 0.6562496325689741, 1.3628938549143559, 0.7079280396869445, 0.46538112321381275, 1.0811587862047156, 0.7498733978794169, 0.09157228346770543, 1.2036515302083237, 0.5837811085099117, 0.5452901141536952, 0.5067802007527524, 0.6483433666613317, 0.6348904925260734, 0.3334779778761936, 0.34246744460938755, 1.0359913422536358, 0.24040898126210755, 1.4388527517473557, 1.6688787403523362, 0.32739502134271986, 0.023937065390097333, 0.21494457861272054, 0.30051632001232476, 0.6393733007980842, 1.1603665874558569, 0.6555790956225527, 0.44284274170133014, 1.2434506156634246, 0.4183415411619176, 0.26268542971135433, 0.49286907173710953, 0.39533411502171173, 0.49786293934932635, 0.4084787122953135, 1.8432184232521038, 1.0444421357835005, 0.16535207465813667, 0.7388199072655669, 1.4330902129990806, 0.438749436598196, 0.5030202150358313, 1.197083043276447, 2.472230413568573, 0.6204502365973841, 0.8385254915624212, 0.6816154409746011, 0.5930563586412285, 0.8758452569712019, 1.1514621947551789, 0.4199567077254943, 0.5953924651018015, 0.3693961434591178, 1.0190779976418771, 0.7258901458941636, 0.5562471336775497, 0.3467229971240643, 0.39176050238971116, 0.3511023446303922, 1.083348408369079, 1.142881120702706, 7.951266665139676, 1.1368683772161603e-13, 1.555057389471424, 0.6594979462374291, 0.49966012323492465, 0.170861330043721, 0.7078920193865202, 0.0, 0.32810717911629494, 0.5800000000000012, 0.5447341129140679, 0.9733326991372867, 1.3512423589509706, 0.2932368274098105, 0.7284992183325364, 0.2902331024750524, 0.1301041249666259, 0.8677414109204393, 1.4531992961890707, 0.792280463808948, 0.6386472973170435]
for i in range(len(avgy)):
    avgy[i] = avgy[i]/100
    stdy[i] = stdy[i]/10

plt.scatter(np.array(x_mi_new), np.array(y_mi), 3)
#plt.plot(np.array(x_mi_new), np.array(avgy), color='red', linewidth=1)
plt.errorbar(x_mi_new, avgy, stdy, color='red', linewidth=0.5)
plt.show()

In [ ]:
avgy = [75.58575581395348, 72.20625, 74.5091911764706, 73.45454545454545, 74.50403225806451, 76.47426470588235, 70.76875, 70.76909722222223, 78.70833333333333, 82.49161585365853, 77.36583333333333, 83.75595238095238, 90.9225543478261, 94.341796875, 92.70436507936508, 94.38980263157895, 96.05555555555556, 92.66041666666666, 91.09114583333333, 85.91517857142857, 86.55208333333333, 86.921875, 88.45138888888889, 91.08035714285714, 90.99479166666667, 90.8125, 92.0, 92.44444444444444, 92.6875, 92.9, 92.96875, 94.109375, 94.33333333333333, 98.5875, 99.11778846153847, 100.69010416666667, 101.49553571428571, 99.58035714285714, 100.359375, 103.88392857142857, 107.63839285714286, 107.57986111111111, 107.8125, 110.009375, 115.98046875, 114.11458333333333, 114.76041666666667, 112.4875, 115.03645833333333, 114.128125, 112.5859375, 116.46875, 116.9375, 117.60526315789474, 119.53125, 119.43229166666667, 118.50520833333333, 117.1875, 119.00694444444444, 118.23611111111111, 117.17857142857143, 117.375, 117.51041666666667, 116.828125, 117.025, 119.10546875, 123.25, 128.125, 130.94375, 132.38839285714286, 136.09615384615384, 136.623046875, 136.74583333333334, 133.5, 135.14930555555554, 136.35714285714286, 136.96875, 139.03720238095238, 140.2878289473684, 142.1171875, 144.140625, 145.68125, 148.4375, 147.875, 146.203125, 146.3125, 148.1125, 148.70833333333334, 148.71484375, 151.88125, 151.625, 151.625, 151.83705357142858, 149.9375, 149.13541666666666, 152.76704545454547, 154.60416666666666, 155.171875, 156.15625, 156.0625, 156.0, 155.8375, 154.796875, 154.375, 148.5721153846154, 146.625, 145.25, 145.75, 145.5625, 145.3125, 146.7, 148.53571428571428, 149.125, 148.4125, 149.0, 151.609375, 151.265625, 150.97222222222223, 151.9375, 155.5, 155.73958333333334, 164.2130681818182, 160.94230769230768, 158.828125, 158.6875, 156.125, 157.5284090909091, 154.5078125, 158.00390625, 157.9375, 158.9375, 157.75, 156.75, 156.5, 158.2058823529412, 157.69642857142858, 158.125, 158.16826923076923, 161.81666666666666, 164.17083333333332, 165.875, 163.0, 164.42578125, 163.25, 161.61141304347825, 161.3566176470588, 160.03125, 165.25986842105263, 162.0625, 163.52604166666666, 163.64285714285714, 164.44485294117646, 164.40625, 165.125, 166.79017857142858, 165.9375, 165.375, 168.46428571428572, 169.80514705882354, 168.7, 169.1875, 170.21875, 169.72916666666666, 169.5625, 171.56041666666667, 175.65509259259258, 175.765625, 176.3465909090909, 176.6875, 178.62890625, 179.58680555555554, 180.60763888888889, 181.34375, 179.99305555555554, 180.00625, 178.63333333333333, 179.7775, 184.31119791666666, 184.625, 184.9, 184.83522727272728, 186.77083333333334, 189.32954545454547, 188.375, 186.04861111111111, 186.46875, 190.1640625, 192.2375, 192.625, 190.7890625, 190.26388888888889, 190.54166666666666, 191.875, 192.75, 194.4375, 195.0, 195.125, 195.0625, 195.4453125, 194.6875, 195.25892857142858, 197.75, 196.8125, 196.0625, 196.25, 198.05769230769232, 198.77083333333334, 195.9375, 196.8125, 198.43055555555554, 199.0625, 198.95625, 198.75, 199.0625, 199.1015625, 199.0, 198.875, 198.8125, 197.7734375, 196.375, 196.75, 196.71527777777777, 195.85714285714286, 194.16796875, 193.8625, 195.5, 196.0859375, 197.05357142857142, 199.3125, 200.5125, 201.4375, 202.67361111111111, 203.3125, 203.3806818181818, 201.1875, 202.74609375, 206.5, 208.859375, 207.82638888888889, 206.77864583333334, 208.55514705882354, 208.6898148148148, 208.15104166666666, 207.875, 207.4672619047619, 210.43055555555554, 214.33522727272728, 215.7421875, 212.54861111111111, 214.7875, 215.8359375, 218.5078125, 218.82986111111111, 218.25, 219.046875, 218.625, 216.896875, 218.08333333333334, 220.86197916666666, 224.546875, 223.071875, 223.61805555555554, 220.875, 223.03125, 223.96875, 224.375, 223.9375, 226.5625, 226.65625, 227.215625, 224.9375, 225.07954545454547, 225.16875, 225.125, 224.65178571428572, 225.35546875, 224.75, 222.64930555555554, 225.04861111111111, 226.22767857142858, 226.890625, 229.0625, 230.996875, 232.05357142857142, 230.9375, 232.85491071428572, 233.875, 232.47767857142858, 232.91964285714286, 233.70738636363637, 232.8359375, 236.09134615384616, 233.25, 233.83203125, 235.1875, 238.6171875, 239.95486111111111, 239.08564814814815, 241.3125, 240.65, 241.87291666666667, 240.087890625, 238.76, 238.709375, 238.43055555555554, 238.8138888888889, 239.97386363636363, 241.54583333333332, 240.34204545454546, 240.95138888888889, 242.5765625, 242.22916666666666, 241.35416666666666, 241.5975, 242.38888888888889, 242.8840909090909, 243.790625, 245.72625, 246.61647727272728, 245.515, 243.83875, 245.13888888888889, 246.9375, 246.94375, 245.5125, 246.06640625, 246.04125, 246.81944444444446, 250.0925, 248.759375, 247.30208333333334, 248.95, 252.23375, 252.08977272727273, 251.79017857142858, 255.65454545454546, 254.70625, 255.97544642857142, 256.46785714285716, 253.73035714285714, 252.646484375, 253.85, 254.76116071428572, 254.75714285714287, 254.675, 253.6015625, 254.4296875, 252.875, 251.9046875, 252.65555555555557, 254.6153846153846, 256.815625, 255.7140625, 256.2013888888889, 258.0885416666667, 258.5111607142857, 258.8097222222222, 261.0892857142857, 258.4171875, 259.628125, 263.3359375, 264.00625, 264.22222222222223, 263.52777777777777, 261.5982142857143, 263.111328125, 260.66015625, 258.8125, 255.725, 254.88616071428572, 256.146875, 255.1296875, 256.16796875, 253.46527777777777, 254.84722222222223, 251.51785714285714, 252.5984375, 253.22222222222223, 252.66927083333334, 251.140625, 251.89375, 252.11160714285714, 250.85546875, 254.84114583333334, 252.925, 255.6578125, 255.6534090909091, 259.1328125, 259.626953125, 259.26785714285717, 259.65625, 258.80078125, 257.0453125, 259.8125, 258.765625, 257.23579545454544, 255.05034722222223, 256.3863636363636, 254.6, 254.59375, 254.24305555555554, 252.15625, 253.6796875, 254.53571428571428, 253.26785714285714, 253.84635416666666, 255.32386363636363, 254.28794642857142, 255.24330357142858, 264.5980392156869, 269.4666666666665, 270.0320512820514, 266.7897435897437, 262.80606060606044, 266.0369047619051, 262.1000000000001, 262.53333333333353, 263.0444444444445, 264.3400000000001, 259.8592592592594, 266.0722222222221, 266.87826086956466, 269.9089743589744, 271.6866666666668, 271.72407407407417, 271.1375000000002, 274.0456790123458, 273.0124999999995, 269.6249999999998, 269.8166666666668]
stdy = [2.032754284479767, 1.077486223809845, 2.2241226558918816, 2.153566497005559, 1.653665854102313, 2.0314687465393333, 0.9474678424622125, 0.85528628913433, 4.120150505864103, 2.001518149870457, 2.482159118366284, 1.6791465158055596, 2.3154426335881477, 1.1981415026209277, 1.4984369480735509, 1.504204237687584, 0.7916362079910628, 0.9300654285526847, 2.6657904138952806, 0.2605016868016993, 0.4003308093869924, 0.6177311687740874, 1.49316860037164, 0.09278843611976129, 0.1830342019875581, 0.15811388300841897, 0.3612045819199972, 0.7605989761162815, 0.701560760020114, 0.3657184709581949, 0.40625, 0.24356964481437338, 0.55979286843935, 0.5736396952791883, 0.263646148824406, 0.5313967832351254, 0.5316250176945924, 0.8931249598334776, 0.15546678704790937, 1.8308579454251854, 0.6818675818915971, 0.8998952378156082, 0.4050462936504913, 1.9937671433432238, 1.010617160896468, 0.1854634772371339, 0.16304662335935965, 1.4604954210814904, 0.3675105700482943, 1.288239320497166, 0.39397005259759277, 1.016369919123938, 0.0, 0.8730129297351584, 0.21875, 0.3797085756610661, 0.5391316883939911, 1.008944374086104, 0.5761796809080982, 0.7317085325067694, 0.45060546003376933, 0.0, 0.16304662335935965, 0.18944305707784595, 1.1809688818931683, 0.5857682047093692, 2.236747235260266, 0.0, 0.6545812115391031, 1.1650075134425317, 0.8437465757107808, 0.5054595427581068, 0.6995037923811103, 0.0, 0.8047544507986563, 0.3019916879515672, 0.7686483672655527, 0.8755376243548861, 0.9100746736012771, 0.556774243831151, 0.5122998847110938, 2.9432722542265775, 0.0, 0.0, 0.7027776920015318, 0.2651650429449553, 0.525, 0.25108272485820726, 0.5270832690011489, 0.517619249545455, 0.1875, 0.0, 0.7706131758365926, 0.0, 0.3825909931709782, 1.6836215761180562, 0.5073248192452466, 0.20432338797846908, 0.06987712429686843, 0.0, 0.0, 0.2, 0.7307089344431201, 0.879452954966893, 2.3921252713036307, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45868357829771933, 0.680448531544916, 0.0, 0.030618621784789725, 0.0, 0.981145496027475, 0.20669932117692114, 0.09820927516479827, 0.0, 0.0, 0.3651780863116271, 1.9118220622285866, 0.19985201625794738, 0.12401959270615269, 0.0, 0.16237976320958225, 0.9775039367086925, 1.0128266737669136, 0.5678859980981548, 0.5537514108334172, 0.0, 0.5031152949374527, 0.0, 0.0, 0.07953421931170565, 0.4799885734184147, 0.0, 0.5062022860851872, 0.5251322584729384, 0.44631982802569836, 0.0, 0.0, 0.43313601965599385, 0.15309310892394862, 1.29785521779265, 0.24607647839237515, 0.7686483672655527, 0.2614666045952323, 0.0, 1.1228156842349009, 0.12371791482634838, 0.39868946210958706, 0.16237976320958225, 0.5, 0.36413359043580895, 0.3535533905932738, 0.0, 0.8130884839013902, 0.3045928702530585, 0.400975373308636, 0.1767766952966369, 0.3788861141556919, 0.2845104225780763, 0.0, 0.7092181973052363, 1.3528701454003267, 0.7810937343718742, 0.2971976552414095, 0.75, 1.601500570448989, 0.2518498306359526, 0.23367307442210056, 0.06987712429686843, 0.6158669113283615, 0.31842303491843466, 0.36505326856349185, 0.23323807579381203, 1.2012966074013498, 0.0, 0.6434768838116876, 0.6419700277496806, 1.0248644896874037, 0.18672359914948844, 0.0, 0.6678230711206281, 0.9084026158592896, 1.316191794475163, 0.0375, 0.0, 0.5710611524554529, 0.2296921584519615, 0.11785113019775792, 1.403121520040228, 0.0, 0.3535533905932738, 0.0, 0.0, 0.0, 0.09077304717673633, 0.0, 1.3030820999398591, 0.0, 0.7071067811865476, 0.0, 0.0, 0.7477777728932147, 0.860116473055959, 0.0, 0.0, 0.49690399499995325, 0.0, 0.028641098093474003, 0.0, 0.4592793267718459, 0.2687091175299975, 0.0, 0.0, 0.0, 0.9677098750884741, 0.0, 0.0, 0.09820927516479826, 1.0164463907684076, 0.16641725315734995, 0.225, 0.0, 0.2118037767457181, 0.5905019915638019, 0.5625, 0.8573214099741123, 0.0, 0.9938079899999066, 0.0, 0.773520648861345, 0.6634034720037774, 1.386399611164089, 1.3110110602126894, 0.8179405190527813, 0.1552824984374854, 0.9746091575511893, 1.3073631420674392, 1.3694629127545408, 0.8164799690932751, 0.0, 0.3190498400775571, 2.4401595968347425, 0.5785419118799026, 0.24494399072798254, 0.44172344149414583, 1.1465641608736947, 0.22736925329461324, 1.0562488443041014, 0.979788268022393, 0.5412658773652742, 0.5392323102105436, 0.0, 0.8026506381203469, 0.914058790353346, 1.0581988872525991, 0.8812001315677387, 0.6316134473513052, 0.3637722459363554, 0.5303300858899106, 0.8149463909928725, 0.5385527481129402, 0.0, 0.46770717334674267, 0.0, 0.59375, 1.1022836735046926, 0.0, 0.26191887662763885, 0.3075228650035636, 0.0, 0.5216914699282693, 0.07564420598061361, 0.25, 0.8052935996490985, 0.12422599874998831, 0.6865281860601952, 1.3058798817751194, 0.0, 1.251635648611448, 0.27836530835928386, 0.15309310892394862, 0.7325068509378724, 0.0, 0.24731498186543824, 0.1694077317947346, 0.23032397793987844, 0.053559801565633156, 1.8249926541961266, 0.0, 1.1292711348026379, 0.0, 0.677078325301992, 1.4628732245097844, 0.6180512206811499, 1.3096447116807417, 0.10777097630314633, 0.7939358924931463, 1.11627031570521, 1.2054796763114672, 0.8622452069307199, 0.6246140783824683, 0.8267961181498461, 0.6447659579226154, 0.6019932170160502, 1.2032068033261145, 0.6676930756042603, 0.43983473014730207, 1.216780816562932, 1.1231466214761692, 1.2597842077117811, 0.7194457850695358, 1.2502510078558735, 1.1660242837129653, 0.5458493954379724, 0.5665245717465061, 1.4406248644251562, 0.7782281558129338, 1.2530441328083508, 0.2851684111665819, 0.914146630743668, 0.6950157372318989, 0.2910352866680216, 0.7758674580751534, 1.2139397395909008, 0.5562485955038449, 1.0406343843420705, 0.6022779385614069, 1.1156906493289258, 0.40750958577682567, 0.6117648227440896, 1.8972058788343646, 1.0861900774254571, 0.8110804907652507, 2.387265045803632, 1.6412900134233637, 0.5085807081817928, 0.808525340554246, 0.9088110364646766, 0.24324212854579652, 0.44840277766803377, 0.6534962063139056, 1.105047678199339, 1.4176060572647642, 1.5026019100214134, 0.5036948442199404, 0.6562496325689741, 1.3628938549143559, 0.7079280396869445, 0.46538112321381275, 1.0811587862047156, 0.7498733978794169, 0.09157228346770543, 1.2036515302083237, 0.5837811085099117, 0.5452901141536952, 0.5067802007527524, 0.6483433666613317, 0.6348904925260734, 0.3334779778761936, 0.34246744460938755, 1.0359913422536358, 0.24040898126210755, 1.4388527517473557, 1.6688787403523362, 0.32739502134271986, 0.023937065390097333, 0.21494457861272054, 0.30051632001232476, 0.6393733007980842, 1.1603665874558569, 0.6555790956225527, 0.44284274170133014, 1.2434506156634246, 0.4183415411619176, 0.26268542971135433, 0.49286907173710953, 0.39533411502171173, 0.49786293934932635, 0.4084787122953135, 1.8432184232521038, 1.0444421357835005, 0.16535207465813667, 0.7388199072655669, 1.4330902129990806, 0.438749436598196, 0.5030202150358313, 1.197083043276447, 2.472230413568573, 0.6204502365973841, 0.8385254915624212, 0.6816154409746011, 0.5930563586412285, 0.8758452569712019, 1.1514621947551789, 0.4199567077254943, 0.5953924651018015, 0.3693961434591178, 1.0190779976418771, 0.7258901458941636, 0.5562471336775497, 0.3467229971240643, 0.39176050238971116, 0.3511023446303922, 1.083348408369079, 1.142881120702706, 7.951266665139676, 1.1368683772161603e-13, 1.555057389471424, 0.6594979462374291, 0.49966012323492465, 0.170861330043721, 0.7078920193865202, 0.0, 0.32810717911629494, 0.5800000000000012, 0.5447341129140679, 0.9733326991372867, 1.3512423589509706, 0.2932368274098105, 0.7284992183325364, 0.2902331024750524, 0.1301041249666259, 0.8677414109204393, 1.4531992961890707, 0.792280463808948, 0.6386472973170435]
for i in range(len(avgy)):
    avgy[i] = avgy[i]/100
    stdy[i] = stdy[i]/10

plt.scatter(np.array(x_mi_new), np.array(y_mi), 3)
plt.plot(np.array(x_mi_new), np.array(avgy), color='red', linewidth=1)
plt.scatter(np.array(x_mi_new), np.array(stdy), 3, color='green')
#plt.errorbar(x_mi_new, avgy, stdy, color='red', linewidth=0.5)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

avgy = [90.09905660377359, 86.61574074074075, 89.11162790697675, 86.43119266055047, 91.95299145299145, 89.01298701298701, 77.11848341232228, 81.08411214953271, 91.7381974248927, 88.29795918367347, 78.03305785123968, 78.30656934306569, 87.31338028169014, 93.75494071146245, 96.3687707641196, 103.02083333333333, 94.49683544303798, 105.79701492537313, 98.35, 100.30952380952381, 100.18075117370893, 104.09684684684684, 106.45951417004049, 111.4718875502008, 117.47610294117646, 124.1824, 121.86988304093568, 135.4850455136541, 145.01824401368302, 154.49631966351208, 165.70806302131604, 178.54784081287045, 176.81904012588512, 176.55329949238578, 184.33216783216784, 192.5694618272841, 197.28938156359393, 207.42895732036737, 214.07221115537848, 227.1863325740319, 240.92993079584775, 249.7665857605178, 266.67291507268556, 269.92119866814653, 280.43077462320366, 288.40969162995594, 295.8191906005222, 294.2707037643208, 295.85641025641024, 289.260490639122, 297.01718103722555, 301.0884844681519, 304.3004413619168, 301.07887234716503, 299.30285530959253, 295.33605220228384, 299.59149484536084, 300.60469604374396, 300.205705229794, 298.9310344827586, 293.28196721311474, 293.5444957152274, 284.5365526492287, 289.810765630224, 292.09451421017843, 293.736490993996, 288.8023295649195, 290.0499001996008, 292.2996031746032, 283.57414965986396, 288.0706042090971, 282.8520689655172, 284.03986135181975, 293.8123947457056, 294.1322786940424, 285.80601092896177, 289.8806837606838, 275.7316814159292, 290.5265831357941, 282.3887741046832, 287.4514664804469, 294.21960382513663, 294.635413153457, 295.7104736490994, 285.92595131984916, 297.2320460704607, 288.1078697421981, 286.1037542662116, 288.7952512044047, 294.641034139874, 291.97697594501716, 286.2924037460978, 294.6948752528658, 294.4008152173913, 291.4129692832764, 292.4946164199192, 288.9413793103448, 295.08152173913044, 300.0609271523179, 300.86490993996, 296.498, 294.32775359892867, 301.72533595542444, 299.06890920966686, 299.4136005256242, 300.3018929503916, 306.07867360208064, 305.4421460892049, 304.7540505508749, 304.91498511412505, 298.6553876478318, 295.0006837606838, 303.68447246184473, 294.55498458376155, 293.60165403170225, 295.3153976311337, 295.80801616705963, 296.61830508474577, 296.6335593220339, 289.58813501534263, 290.27508532423207, 292.14160189253124, 284.60130494505495, 291.4056767047421, 292.2325180847399, 288.6061231510148, 290.89866484080795, 294.65868673050613, 292.46634779637856, 287.16438836312653, 292.69972826086956, 284.54592541436466, 294.15570934256056, 293.86380660537964, 287.0396551724138, 291.2917093142272, 293.8023529411765, 292.60841533763147, 289.46494845360826, 289.24625340599454, 286.64518364518364, 285.93385082686467, 287.1222826086956, 285.71809986130376, 293.62323232323234, 291.4010803511141, 291.7685867381112, 285.3119105833042, 288.5318118948824, 291.57074829931975, 291.25359652057546, 291.8068257123923, 284.079642979746, 287.29218967921895, 293.74594594594595, 288.3913340156943, 286.5679809653297, 291.37295081967216, 280.4504847645429, 278.3402922755741, 290.49321113374066, 282.99895470383274, 281.121736062103, 286.57885211995864, 257.501776549546, 247.7256733802475, 244.94449909821282, 228.92823377739577, 224.79154289708194, 229.6123595505618, 225.72794759825328, 229.92055870798777, 232.70471249459575, 235.86905790838375, 239.64708368554523, 246.16883116883116, 249.96461600321672, 256.125430210325, 258.46460348162475, 258.25890207715133, 269.2150183150183, 267.0948905109489, 264.41755416819683, 272.0596529284165, 264.54824561403507, 271.5508173418621, 270.827289048474, 272.4322278298486, 269.8796965317919, 263.891280148423, 268.05139500734214, 275.45099457504523, 275.53136265320836, 273.7152949487814, 274.79666430092266, 273.6883537023724, 265.6896551724138, 265.61452095808386, 261.92936802973975, 278.52116788321166, 269.66097289268475, 263.9430591737998, 271.31107795598655, 264.8927231807952, 267.63274336283183, 266.774217585693, 268.51707498144026, 266.48067542213886, 273.10348706411696, 269.11674748228273, 272.1484667165295, 273.09399189089567, 272.97861163227014, 261.6545866364666, 265.60853293413174, 266.3278504672897, 273.8628026412326, 276.47617328519857, 281.0676015821647, 275.1092739036664, 282.04831782390835, 269.2145441389291, 276.36883771140697, 273.46010064701653, 267.4241654571843, 266.18560885608855, 273.5263535317318, 264.5532786885246, 267.5994009734182, 269.9882604055496, 278.5210355987055, 272.86027798098024, 263.9610632721827, 263.1833962264151, 272.1483536810951, 270.6580813736469, 266.5460258780037, 256.43769113149847, 255.65633475397132, 261.66959847036327, 255.84108231707316, 258.76351092372556, 257.9684857801691, 259.50233281493, 252.14703557312254, 259.0302681992337, 261.34685367702804, 261.4275389882084, 261.18514285714286, 257.12047244094487, 272.28351309707244, 263.6768361581921, 257.705378020265, 259.2112403100775, 254.27394560504533, 257.7723196881092, 252.03094010313367, 256.02877138413686, 255.17241379310346, 249.2498990714574, 249.20631747301078, 255.85852090032154, 257.86072423398326, 253.13181448331977, 247.62520798668885, 256.44833197721726, 258.8162614587485, 253.13498175922172, 250.25236722931248, 260.7946966653274, 262.8992248062016, 255.1661316211878, 257.534219788815, 265.33437256430244, 255.11001577287067, 249.16789215686273, 254.53343823760818, 252.83965728274174, 250.93495610534717, 251.9419407226959, 253.51278928136418, 255.705174488568, 251.35238095238094, 241.82012934518997, 252.35115431348723, 249.7523120225171, 248.41611570247935, 249.6513687600644, 259.45081967213116, 244.99103504482477, 258.18028940164254, 251.59683313032886, 255.48741883116884, 259.1633935585232, 255.5207336523126, 255.27174780526735, 248.7858306188925, 252.3637459807074, 240.08033826638479, 237.41486603284355, 241.7835314091681, 237.88846153846154, 243.30461922596754, 262.2184710904152, 259.43265465935787, 252.12925170068027, 248.80213903743316, 252.46474878444084, 253.17139432680784, 258.3652412710867, 251.55025620811983, 250.76534296028882, 253.91975806451612, 254.0178855325914, 254.5203798971112, 257.3195384003183, 254.52988691437804, 253.8014240506329, 244.8312706270627, 248.97022838499186, 249.95962858296326, 255.32073954983923, 254.7171235597934, 247.63557929334428, 254.5983436853002, 248.04603370324702, 239.35799319727892, 241.5031605562579, 244.71867219917013, 247.9946258784622, 256.76307320997586, 248.4325, 244.706132879046, 241.9325513196481, 240.46463362981788, 241.2685146876293, 245.3185618729097, 247.75644222776393, 245.55046649703138, 245.70210135970333, 255.8832731648616, 254.8659035159444, 247.09376308078694, 251.64446260727422, 252.02837454398053, 254.73927125506074, 252.02157088753498, 256.0, 254.11998064828254, 255.29870654810026, 244.93246573991917, 253.76833660772758, 256.96048513302037, 262.6106117943866, 256.789702970297, 257.45438229135505, 261.8406795584436, 257.7343870470316, 250.88662440824842, 250.41332263242376, 256.9209555253447, 256.65607050121963, 259.0311284046693, 253.00981668837022, 262.33900676148284, 258.9112426035503, 267.2485017978426, 262.51095732410613, 263.4932144241954, 257.90728213290885, 257.62640807130964, 256.98991967238936, 254.48973954222572, 257.1126056879324, 267.3668770887486, 256.94528781690684, 255.69535151747982, 262.4707916986933, 266.318356374808, 264.6378388518901, 258.99314540973506, 260.3851794673871, 266.6387394312068, 263.98035511900264, 258.9147137917787, 254.83950136345928, 259.8260372178157, 250.99646226415095, 264.0266101041265, 264.5378119001919, 257.42160686427457, 261.77798789712557, 255.75548710050057, 262.25349981082104, 267.5142428785607, 273.5185860058309, 263.9199401421624, 268.20620097123646, 261.80994287478774, 261.69275250578255, 262.1546940326872, 255.43139580862, 264.03914044512663, 261.6606129398411, 266.3211397766654, 251.26288453855375, 254.43364928909952, 255.5406237966885, 260.6522247103495, 267.99690285541624, 261.36664947124063, 265.6770833333333, 268.8351483289523, 270.5515947467167, 274.73127753303964, 272.1833768169959, 266.6215520411916, 272.5371870397644, 276.31673544245757, 272.272169753465, 269.07441348973606, 266.24329224075416, 277.1734290843806, 282.79016681299385, 279.838766211006, 274.7250268528464, 273.694021931376, 275.17315864022663, 273.7820895522388, 229.3030039181541]
stdy = [41.08965046091291, 33.811749067446954, 35.01467326350483, 34.20977336595494, 33.07041493162733, 32.065302021811675, 30.109630415367107, 32.5460611096028, 34.034392072578584, 34.59040464854379, 33.473870406632095, 30.091462140447746, 36.24692874920577, 39.81519810323691, 42.05428040045389, 43.60171298240471, 44.01131779156357, 53.07600682040803, 40.04509177165161, 43.30108976688693, 41.62589150696955, 44.49139664024945, 45.566872218202555, 45.09707411517958, 45.202322949158265, 49.58814752539159, 46.870791477742294, 51.79828665120112, 57.742389920664, 55.42548488736928, 60.651154152250065, 63.596844212193425, 63.62681112353502, 61.788417957979796, 61.93645937571827, 65.5141921647516, 65.73357090368665, 71.75715883276037, 73.64229423368069, 80.82842553829686, 85.83302740641274, 89.28386962380159, 93.42290855967823, 92.19985086755835, 98.19843987431776, 102.77402395547233, 101.12938417566875, 100.45015866090009, 99.51915644738956, 98.21715417889077, 94.34237846800345, 99.72642265217378, 97.64700456235204, 100.61428514381046, 100.28239423622674, 98.45618870213289, 102.59901848336563, 103.78643938395805, 101.33297140491835, 102.10271712226859, 98.76036338025493, 101.43771179891628, 99.16508883548686, 100.70594570331983, 99.93028820132562, 102.55912928647551, 99.52478884051952, 100.04467751959622, 98.05005099437736, 96.02018017062872, 95.18239070332285, 92.0132463289933, 96.4040974232942, 102.2093248864799, 100.40449442131562, 95.55692020989375, 93.79196977220384, 88.88607525928907, 93.67263474462538, 85.08155554378399, 87.75086805126203, 99.06618683991022, 98.24298241477607, 100.82340688054535, 93.38275988505588, 97.94697226050431, 93.41421114450414, 93.00653118867513, 94.15510391768467, 98.40618549537422, 99.4660746184346, 97.93269372380196, 98.66982504798516, 99.91684042242329, 95.43903485995654, 98.01863263558617, 99.62735254436906, 102.31683329247441, 103.7925929137046, 105.16594030388356, 100.89962524707414, 96.24634733328844, 100.53349877031039, 99.45405660404205, 96.86635042030849, 100.58764834710075, 102.04586897567192, 103.92658165743738, 106.65921197908786, 107.06218987112116, 102.39232148945545, 101.85342946484424, 106.118374351476, 99.80432026595842, 97.17373294774674, 97.95335305644213, 100.55205042041659, 100.57894908975734, 99.93839899533111, 87.4558594020435, 86.12772129227616, 73.96702707118895, 81.1660864755102, 84.61309000385224, 82.46284674779304, 88.43074179124588, 90.97149278757604, 97.04845358762807, 92.31448880728311, 95.3217413552648, 96.49301707248043, 95.79796448776978, 98.36570794743461, 96.42320049526462, 88.69602231780183, 100.33226051475168, 101.44316065536404, 102.32118548961402, 95.59499120639114, 91.3215472930677, 83.63814825634293, 87.55605347433152, 87.51062435498903, 87.50428560933459, 92.95594974906125, 92.79008567729636, 93.40384009102624, 85.59294564244182, 92.72573806662312, 90.11095590992252, 99.7803838437195, 95.28491404073365, 92.25702886501385, 98.93045221265291, 95.28870604641455, 93.81622660819396, 91.5110777105701, 89.65853190717546, 88.56616082765471, 90.21917756220127, 96.33859558868397, 93.7679982723317, 92.9095662458931, 96.09270832768738, 83.76266228905335, 78.87955612831502, 80.31634232053898, 72.50075430642084, 69.91152332591531, 71.74670615261721, 75.14462618012016, 82.8106839423513, 80.49454076985594, 78.97062349380306, 80.37487849913056, 78.83960675796145, 82.6111940054131, 81.16396887628154, 83.41827962143549, 86.13250602269737, 83.66037648582511, 86.3217093204253, 79.89201696665319, 86.0703745489701, 84.73082378922089, 89.4908340278489, 84.3065231387821, 90.11483559187133, 85.01541680042509, 86.37977103321124, 85.91872522186301, 88.51129871378004, 91.8471684035496, 90.12801225895309, 89.40250887838663, 89.3757648568671, 87.68614129382134, 86.51508177768775, 83.89725859645236, 92.1679640317068, 92.90248163396929, 85.98182947576773, 88.97813987154373, 82.09893041325203, 87.99422921419335, 84.40487896887241, 89.38328195893234, 86.78221213906684, 88.4056479742669, 87.89604566617318, 91.0913003529975, 90.38355768611899, 94.69504920401066, 83.18869739934627, 84.47057087382564, 87.02601532731462, 87.47705056184736, 88.00843354474615, 94.01792963977668, 87.31818163332308, 95.07872971253876, 88.7094757833119, 91.9652617840019, 87.12912472302244, 84.71032902633539, 87.77597104988358, 88.67582727976097, 81.53792733292894, 88.51474100397063, 87.76368750086792, 92.16823951340288, 90.87472059791986, 83.16850365372699, 84.05838744587003, 90.85832033996667, 86.95688586880283, 87.09349771222878, 83.40223974060889, 80.79949624688263, 89.25306367290705, 81.39331111184751, 85.11662495511673, 84.21306219197827, 86.94104953213989, 84.52160873853502, 86.59739819994594, 84.87746222496287, 81.85419583014667, 86.80813523944632, 84.60937355724838, 92.03716555690967, 86.75123828366948, 85.80371404403192, 85.65736158527181, 82.4516667129901, 81.47428179339785, 80.87526564871364, 83.304575767181, 80.79968962656973, 81.52489554577792, 81.09983719928172, 84.55388444654686, 89.06315789230696, 83.39654893938958, 83.34107255729315, 85.62089406213883, 84.15304138888861, 81.17948863475306, 78.96112572019221, 88.81863543198578, 91.5107362212216, 83.35382039834766, 86.60937359056466, 87.44388825841403, 82.95518368372166, 81.32974452037827, 87.20806209147179, 85.87345340674264, 81.89245997037823, 83.36291141748829, 84.4748483277715, 85.61687917694735, 80.03610025326071, 72.53878273034364, 83.87860927405747, 74.74704739151909, 77.87369581572457, 83.14294918993431, 79.31385440135917, 76.47260028108107, 81.62327716405413, 79.35282662734832, 81.17417134624782, 86.39297479396112, 82.76904520259974, 82.1492544092763, 78.19364484246786, 81.49134827544579, 79.14138850816303, 71.94548174659754, 77.05738020300197, 79.19297534346086, 78.52047404180645, 84.47778461672631, 78.54686101302839, 81.77102176199097, 77.66021745398348, 84.84468807032059, 84.17854443829496, 84.00425491455775, 81.56889342758059, 79.03537083958042, 81.08627735319953, 84.2926449934987, 81.68362653402455, 76.49662982772509, 83.75559682791354, 72.76320756398799, 75.75855974739753, 81.84398267435427, 80.65123119797985, 82.13058866068111, 83.13392809196496, 84.01374050564586, 88.669681966273, 82.07616032533441, 81.87030043764344, 81.2425958164804, 81.11024037536814, 82.70464674823296, 84.40450872879956, 82.42721694774123, 82.21710493413156, 79.39773296511683, 77.92343999080123, 81.4308564059104, 79.94096943213786, 82.35251969429957, 77.35460151148088, 83.9790710534476, 83.90880466315797, 83.05549255618197, 79.78582169627634, 85.2838779825941, 80.38090472089749, 80.84205279432234, 78.68678017939227, 82.25966413589347, 84.094348056216, 83.85990922365704, 81.42087800435463, 80.03980943099502, 84.10742294084393, 84.4596797294425, 85.09918386065756, 86.653674907646, 92.01848914076996, 83.12074707165473, 79.95269597549542, 81.84243149949786, 81.50373841119192, 87.0178681363776, 83.40029601266413, 84.55388444654686, 90.7426822118456, 90.53374716093441, 94.2495855428553, 85.95165792467299, 89.18097383270717, 91.36138321933397, 84.27445398072894, 85.98834859299252, 82.3305077112974, 87.55855183818426, 89.96420729240046, 87.58317475405879, 78.76525149931281, 85.78513766235967, 77.84319173826314, 91.32009294235306, 84.8497753385358, 86.99602631011373, 90.7255261420401, 88.30548892905809, 87.02727212632831, 83.83724466935921, 82.30447948775328, 78.79562170577753, 82.80926880488681, 88.65514296982437, 77.76665316027173, 83.77031003732766, 81.22574157094536, 87.68614129382134, 81.74350127074322, 91.10021953870364, 89.33816985896901, 93.77730852263782, 87.82953571407513, 85.8398763687367, 85.51295132171501, 85.10726228560051, 86.52736200185466, 84.21020944636108, 88.94746957474395, 76.41393749670279, 79.58368237270753, 84.47188881515554, 82.83509125817392, 84.37527777732053, 89.96715980706516, 86.32747878717413, 89.3259480778122, 91.77313655830882, 87.78362514586647, 89.72771311027603, 90.91135792627894, 90.33926938906468, 94.00856011555544, 95.26246637579777, 80.79756242455585, 83.40860923040259, 84.91648010692624, 89.90446318175755, 93.16263920558498, 83.72835541201081, 82.57875029328065, 87.32482464912255, 92.26862955522857, 78.53572037614477]
for i in range(len(avgy)):
    avgy[i] = avgy[i]/100
    stdy[i] = stdy[i]/100

plt.scatter(np.array(x_mi_new), np.array(y_mi), 3)
plt.plot(np.array(x_mi_new), np.array(avgy), color='red', linewidth=1)
plt.plot(np.array(x_mi_new), np.array(stdy), linewidth=1, color='green')
#plt.errorbar(x_mi_new, avgy, stdy, color='red', linewidth=0.5)
plt.show()

In [ ]:

avgy = [71.125, 74.125, 75.8125, 71.0625, 77.1875, 73.9375, 67.3125, 70.5625, 75.3125, 72.5625, 65.0, 68.4375, 72.6875, 78.0, 77.75, 84.875, 79.4375, 85.25, 79.125, 80.375, 80.6875, 81.875, 84.3125, 89.4375, 92.5, 99.875, 101.4375, 107.75, 114.0625, 123.625, 130.25, 142.0625, 140.5625, 141.0625, 149.5, 156.875, 160.8125, 167.6875, 174.75, 184.875, 197.0625, 205.375, 220.3125, 224.75, 233.9375, 240.0, 248.1875, 248.875, 250.25, 244.875, 254.625, 256.625, 259.75, 255.875, 252.9375, 251.5625, 253.0625, 252.5, 254.5625, 252.3125, 248.875, 246.125, 238.3125, 242.75, 245.75, 246.5, 243.0625, 243.25, 247.75, 239.0, 243.75, 240.25, 240.0, 245.5625, 246.75, 242.5, 246.9375, 235.125, 248.75, 244.4375, 248.6875, 248.375, 249.0625, 249.625, 243.6875, 251.875, 246.3125, 244.3125, 246.0625, 250.1875, 245.5, 241.75, 250.625, 248.0, 247.875, 248.1875, 242.375, 247.625, 252.8125, 253.0, 251.125, 251.625, 256.125, 253.875, 257.3125, 255.0, 260.125, 258.625, 255.75, 255.75, 252.75, 248.4375, 254.125, 248.8125, 248.375, 250.125, 249.6875, 250.5, 249.9375, 249.8125, 250.625, 260.5625, 248.9375, 252.0, 254.5625, 247.5625, 248.25, 249.1875, 250.3125, 242.375, 247.8125, 238.5, 246.75, 248.0625, 246.625, 244.25, 247.6875, 244.5, 244.8125, 246.5, 249.6875, 245.75, 247.375, 246.5, 252.0625, 249.0, 248.8125, 246.1875, 245.75, 251.625, 245.0, 249.6875, 240.875, 240.875, 250.25, 245.375, 245.1875, 250.8125, 239.6875, 236.0, 246.0, 238.75, 239.25, 243.9375, 218.9375, 214.875, 208.6875, 197.375, 191.5625, 197.6875, 191.3125, 190.875, 194.5625, 199.625, 201.5625, 210.0625, 212.375, 219.6875, 221.625, 221.0625, 231.9375, 229.25, 230.125, 234.125, 227.25, 231.875, 234.3125, 232.9375, 233.5625, 225.6875, 230.1875, 236.5, 234.8125, 234.9375, 234.0625, 233.1875, 225.375, 226.125, 223.5, 235.9375, 226.5625, 224.5, 230.875, 227.375, 228.125, 228.9375, 228.5625, 226.8125, 233.0625, 229.6875, 230.0, 233.25, 229.8125, 223.375, 227.1875, 226.8125, 234.625, 237.375, 238.5625, 235.6875, 238.6875, 230.4375, 234.375, 235.125, 229.6875, 226.4375, 233.1875, 229.0625, 227.875, 231.3125, 237.375, 233.3125, 227.0, 225.25, 231.125, 231.5, 227.8125, 219.0625, 220.0625, 221.375, 220.5625, 220.6875, 220.6875, 220.4375, 214.1875, 220.125, 223.0625, 225.125, 222.8125, 219.4375, 233.6875, 224.1875, 218.1875, 219.9375, 216.8125, 220.0625, 215.9375, 218.1875, 218.3125, 211.0625, 211.9375, 217.125, 216.5625, 214.875, 208.625, 216.25, 218.625, 214.875, 213.625, 220.0, 219.3125, 215.375, 216.0625, 223.9375, 213.75, 211.1875, 212.4375, 212.0, 212.5, 212.5, 214.0, 215.5, 214.1875, 208.5, 212.5625, 215.4375, 213.75, 211.5, 223.25, 209.9375, 219.625, 214.4375, 217.5625, 218.4375, 217.6875, 217.5, 213.4375, 214.3125, 203.875, 204.8125, 206.6875, 201.8125, 206.6875, 223.5625, 223.625, 215.0, 214.375, 212.5625, 214.8125, 219.6875, 213.125, 214.3125, 216.875, 214.5, 217.3125, 222.625, 216.0, 221.125, 210.125, 211.25, 214.5625, 217.9375, 217.0, 208.0625, 213.25, 209.4375, 202.4375, 203.125, 207.4375, 210.0625, 217.4375, 209.4375, 207.1875, 205.5, 204.25, 204.875, 209.0625, 209.75, 209.875, 206.875, 216.75, 217.6875, 210.1875, 211.6875, 215.3125, 217.75, 215.625, 218.8125, 216.125, 216.625, 208.125, 217.5625, 219.0625, 223.75, 218.4375, 218.125, 219.8125, 219.9375, 215.0625, 212.9375, 220.875, 216.375, 221.125, 214.375, 220.125, 217.375, 224.625, 222.25, 221.4375, 215.1875, 218.0625, 217.4375, 215.75, 216.5, 225.9375, 216.25, 220.6875, 221.6875, 230.75, 222.375, 219.125, 220.9375, 224.4375, 224.125, 219.4375, 217.0625, 222.8125, 216.0, 226.0, 223.625, 222.1875, 223.6875, 218.5625, 221.875, 230.0, 233.0, 223.0625, 224.9375, 221.1875, 222.375, 223.6875, 216.4375, 224.875, 223.875, 224.8125, 216.6875, 218.75, 217.0, 224.1875, 230.25, 220.6875, 227.0625, 227.0, 227.9375, 235.3125, 231.75, 226.0, 231.9375, 233.625, 229.25, 234.4375, 229.5625, 239.0625, 242.25, 238.1875, 236.75, 237.0, 237.5, 228.0, 192.625, 208.73333333333332]
stdy = [41.08965046091291, 33.811749067446954, 35.01467326350483, 34.20977336595494, 33.07041493162733, 32.065302021811675, 30.109630415367107, 32.5460611096028, 34.034392072578584, 34.59040464854379, 33.473870406632095, 30.091462140447746, 36.24692874920577, 39.81519810323691, 42.05428040045389, 43.60171298240471, 44.01131779156357, 53.07600682040803, 40.04509177165161, 43.30108976688693, 41.62589150696955, 44.49139664024945, 45.566872218202555, 45.09707411517958, 45.202322949158265, 49.58814752539159, 46.870791477742294, 51.79828665120112, 57.742389920664, 55.42548488736928, 60.651154152250065, 63.596844212193425, 63.62681112353502, 61.788417957979796, 61.93645937571827, 65.5141921647516, 65.73357090368665, 71.75715883276037, 73.64229423368069, 80.82842553829686, 85.83302740641274, 89.28386962380159, 93.42290855967823, 92.19985086755835, 98.19843987431776, 102.77402395547233, 101.12938417566875, 100.45015866090009, 99.51915644738956, 98.21715417889077, 94.34237846800345, 99.72642265217378, 97.64700456235204, 100.61428514381046, 100.28239423622674, 98.45618870213289, 102.59901848336563, 103.78643938395805, 101.33297140491835, 102.10271712226859, 98.76036338025493, 101.43771179891628, 99.16508883548686, 100.70594570331983, 99.93028820132562, 102.55912928647551, 99.52478884051952, 100.04467751959622, 98.05005099437736, 96.02018017062872, 95.18239070332285, 92.0132463289933, 96.4040974232942, 102.2093248864799, 100.40449442131562, 95.55692020989375, 93.79196977220384, 88.88607525928907, 93.67263474462538, 85.08155554378399, 87.75086805126203, 99.06618683991022, 98.24298241477607, 100.82340688054535, 93.38275988505588, 97.94697226050431, 93.41421114450414, 93.00653118867513, 94.15510391768467, 98.40618549537422, 99.4660746184346, 97.93269372380196, 98.66982504798516, 99.91684042242329, 95.43903485995654, 98.01863263558617, 99.62735254436906, 102.31683329247441, 103.7925929137046, 105.16594030388356, 100.89962524707414, 96.24634733328844, 100.53349877031039, 99.45405660404205, 96.86635042030849, 100.58764834710075, 102.04586897567192, 103.92658165743738, 106.65921197908786, 107.06218987112116, 102.39232148945545, 101.85342946484424, 106.118374351476, 99.80432026595842, 97.17373294774674, 97.95335305644213, 100.55205042041659, 100.57894908975734, 99.93839899533111, 87.4558594020435, 86.12772129227616, 73.96702707118895, 81.1660864755102, 84.61309000385224, 82.46284674779304, 88.43074179124588, 90.97149278757604, 97.04845358762807, 92.31448880728311, 95.3217413552648, 96.49301707248043, 95.79796448776978, 98.36570794743461, 96.42320049526462, 88.69602231780183, 100.33226051475168, 101.44316065536404, 102.32118548961402, 95.59499120639114, 91.3215472930677, 83.63814825634293, 87.55605347433152, 87.51062435498903, 87.50428560933459, 92.95594974906125, 92.79008567729636, 93.40384009102624, 85.59294564244182, 92.72573806662312, 90.11095590992252, 99.7803838437195, 95.28491404073365, 92.25702886501385, 98.93045221265291, 95.28870604641455, 93.81622660819396, 91.5110777105701, 89.65853190717546, 88.56616082765471, 90.21917756220127, 96.33859558868397, 93.7679982723317, 92.9095662458931, 96.09270832768738, 83.76266228905335, 78.87955612831502, 80.31634232053898, 72.50075430642084, 69.91152332591531, 71.74670615261721, 75.14462618012016, 82.8106839423513, 80.49454076985594, 78.97062349380306, 80.37487849913056, 78.83960675796145, 82.6111940054131, 81.16396887628154, 83.41827962143549, 86.13250602269737, 83.66037648582511, 86.3217093204253, 79.89201696665319, 86.0703745489701, 84.73082378922089, 89.4908340278489, 84.3065231387821, 90.11483559187133, 85.01541680042509, 86.37977103321124, 85.91872522186301, 88.51129871378004, 91.8471684035496, 90.12801225895309, 89.40250887838663, 89.3757648568671, 87.68614129382134, 86.51508177768775, 83.89725859645236, 92.1679640317068, 92.90248163396929, 85.98182947576773, 88.97813987154373, 82.09893041325203, 87.99422921419335, 84.40487896887241, 89.38328195893234, 86.78221213906684, 88.4056479742669, 87.89604566617318, 91.0913003529975, 90.38355768611899, 94.69504920401066, 83.18869739934627, 84.47057087382564, 87.02601532731462, 87.47705056184736, 88.00843354474615, 94.01792963977668, 87.31818163332308, 95.07872971253876, 88.7094757833119, 91.9652617840019, 87.12912472302244, 84.71032902633539, 87.77597104988358, 88.67582727976097, 81.53792733292894, 88.51474100397063, 87.76368750086792, 92.16823951340288, 90.87472059791986, 83.16850365372699, 84.05838744587003, 90.85832033996667, 86.95688586880283, 87.09349771222878, 83.40223974060889, 80.79949624688263, 89.25306367290705, 81.39331111184751, 85.11662495511673, 84.21306219197827, 86.94104953213989, 84.52160873853502, 86.59739819994594, 84.87746222496287, 81.85419583014667, 86.80813523944632, 84.60937355724838, 92.03716555690967, 86.75123828366948, 85.80371404403192, 85.65736158527181, 82.4516667129901, 81.47428179339785, 80.87526564871364, 83.304575767181, 80.79968962656973, 81.52489554577792, 81.09983719928172, 84.55388444654686, 89.06315789230696, 83.39654893938958, 83.34107255729315, 85.62089406213883, 84.15304138888861, 81.17948863475306, 78.96112572019221, 88.81863543198578, 91.5107362212216, 83.35382039834766, 86.60937359056466, 87.44388825841403, 82.95518368372166, 81.32974452037827, 87.20806209147179, 85.87345340674264, 81.89245997037823, 83.36291141748829, 84.4748483277715, 85.61687917694735, 80.03610025326071, 72.53878273034364, 83.87860927405747, 74.74704739151909, 77.87369581572457, 83.14294918993431, 79.31385440135917, 76.47260028108107, 81.62327716405413, 79.35282662734832, 81.17417134624782, 86.39297479396112, 82.76904520259974, 82.1492544092763, 78.19364484246786, 81.49134827544579, 79.14138850816303, 71.94548174659754, 77.05738020300197, 79.19297534346086, 78.52047404180645, 84.47778461672631, 78.54686101302839, 81.77102176199097, 77.66021745398348, 84.84468807032059, 84.17854443829496, 84.00425491455775, 81.56889342758059, 79.03537083958042, 81.08627735319953, 84.2926449934987, 81.68362653402455, 76.49662982772509, 83.75559682791354, 72.76320756398799, 75.75855974739753, 81.84398267435427, 80.65123119797985, 82.13058866068111, 83.13392809196496, 84.01374050564586, 88.669681966273, 82.07616032533441, 81.87030043764344, 81.2425958164804, 81.11024037536814, 82.70464674823296, 84.40450872879956, 82.42721694774123, 82.21710493413156, 79.39773296511683, 77.92343999080123, 81.4308564059104, 79.94096943213786, 82.35251969429957, 77.35460151148088, 83.9790710534476, 83.90880466315797, 83.05549255618197, 79.78582169627634, 85.2838779825941, 80.38090472089749, 80.84205279432234, 78.68678017939227, 82.25966413589347, 84.094348056216, 83.85990922365704, 81.42087800435463, 80.03980943099502, 84.10742294084393, 84.4596797294425, 85.09918386065756, 86.653674907646, 92.01848914076996, 83.12074707165473, 79.95269597549542, 81.84243149949786, 81.50373841119192, 87.0178681363776, 83.40029601266413, 84.55388444654686, 90.7426822118456, 90.53374716093441, 94.2495855428553, 85.95165792467299, 89.18097383270717, 91.36138321933397, 84.27445398072894, 85.98834859299252, 82.3305077112974, 87.55855183818426, 89.96420729240046, 87.58317475405879, 78.76525149931281, 85.78513766235967, 77.84319173826314, 91.32009294235306, 84.8497753385358, 86.99602631011373, 90.7255261420401, 88.30548892905809, 87.02727212632831, 83.83724466935921, 82.30447948775328, 78.79562170577753, 82.80926880488681, 88.65514296982437, 77.76665316027173, 83.77031003732766, 81.22574157094536, 87.68614129382134, 81.74350127074322, 91.10021953870364, 89.33816985896901, 93.77730852263782, 87.82953571407513, 85.8398763687367, 85.51295132171501, 85.10726228560051, 86.52736200185466, 84.21020944636108, 88.94746957474395, 76.41393749670279, 79.58368237270753, 84.47188881515554, 82.83509125817392, 84.37527777732053, 89.96715980706516, 86.32747878717413, 89.3259480778122, 91.77313655830882, 87.78362514586647, 89.72771311027603, 90.91135792627894, 90.33926938906468, 94.00856011555544, 95.26246637579777, 80.79756242455585, 83.40860923040259, 84.91648010692624, 89.90446318175755, 93.16263920558498, 83.72835541201081, 82.57875029328065, 87.32482464912255, 92.26862955522857, 78.53572037614477, 63.29346113321835]

In [ ]:
# output:
# requires:
def find_min_max_vals():
    min_max_vals = {}
    for i in range(len(names)):
        min_max_vals[names[i]] = col_bins[col_means_name[i]]
    return min_max_vals

# global variable
min_max_vals = find_min_max_vals()

# debug
print(min_max_vals)

# user input parameters
# list of feature names with entropy labelings, order corresponding to `data_processing.feature_ref`
features_ent = ['GFP Entropy', 'RFP Entropy']

def entropy_formula(feature_col, min_val, max_val, i):
    feature_ser = pd.Series(feature_col)
    val_counts = feature_ser.value_counts(bins=np.linspace(min_val, max_val, i))
    s = entropy(value_counts, base=2)
    return s

def calculate_entropies(group, min_max_vals):
    group['GFP_entropy'] = entropy_full_sup(group['mean_F_C2'], *min_max_vals['GFP'])
    group['RFP_entropy'] = entropy_full_sup(group['mean_F_C3'], *min_max_vals['RFP'])
    return group
    
        
def add_entropies():
    for i in dfs:
        df = dfs[i]
        dfs[i] = df.groupby('Time_Stamp').apply(lambda group: calculate_entropies(group, min_max_vals))
        
add_entropies()

# debug
print(dfs)